In [ ]:
# %pip install kaggle
import kaggle # Kaggle API for downloading datasets
import os # os for file operations
import zipfile # Extracting zip files
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder # Data handling libraries
import torch # Deep learning library
from torch.utils.data import Dataset, DataLoader # PyTorch utilities for data loading


In [ ]:
# Set the Kaggle API key environmental variable before running the command
# Downloads dataset from Kaggle 

!kaggle datasets download -d arashnic/book-recommendation-dataset

# Unzip the downloaded dataset

with zipfile.ZipFile('book-recommendation-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('book-recommendation-dataset')

# Remove the zip file after extraction
os.remove('book-recommendation-dataset.zip')


Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
  0%|                                               | 0.00/24.3M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 24.3M/24.3M [00:00<00:00, 1.91GB/s]


In [18]:

# Load the dataset into a pandas DataFrame
raw_books = pd.read_csv('book-recommendation-dataset/Books.csv')
raw_users = pd.read_csv('book-recommendation-dataset/Users.csv')
raw_ratings = pd.read_csv('book-recommendation-dataset/Ratings.csv')


/var/folders/lt/bw6r3snx43x5hctrgyvr5vmr0000gn/T/ipykernel_77678/2080199567.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_books = pd.read_csv('book-recommendation-dataset/Books.csv')


In [ ]:
raw_books.head() 
raw_users.head()
raw_ratings.head()  

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
